In [3]:
import pickle
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


In [81]:
# import joblib
# enc=joblib.load("/home/brijesh/Projects/TLT/r/encoders_oct2023.pkl")

##### Save Fitted encoder

In [121]:
def load_encoders(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [122]:
def fit_encoder(df, encoders=None):
    cat_cols1, cat_cols2 = ['Manufacturername', 'ModelYear', 'ModelFamilydesc', 'EngineFuelType'], ['RepairType', 'ServiceType']
        
    # Convert selected columns to string type
    df[cat_cols1 + cat_cols2] = df[cat_cols1 + cat_cols2].astype(str)

    if encoders is None:
        encoder1, encoder2 = OneHotEncoder(sparse=False, sparse_output=False), OneHotEncoder(sparse=False, sparse_output=False)
    
    else:
        if isinstance(encoders, str):
            # Load encoders from a file
            encoders = load_encoders(encoders)
        
        # Use the loaded encoders
        encoder1, encoder2 = encoders['encoder1'], encoders['encoder2']
    
    # Fit the encoders on your data
    fitted_encoder1, fitted_encoder2 = encoder1.fit(df[cat_cols1]), encoder2.fit(df[cat_cols2])
    encoders = {'encoder1': fitted_encoder1, 'encoder2': fitted_encoder2}
    filename = 'encoder.pkl'
    
    with open(filename, 'wb') as file:
        pickle.dump(encoders, file)



##### OneHotEncoding

In [49]:
# def preprocess(df, encoders = None):
#     # Drop unnecessary columns
#     df1 = df.drop(['MajorRepairInd', 'MinorRepairInd'], axis=1)
    
#     # Define categorical columns
#     cat_cols1, cat_cols2 = ['Manufacturername', 'ModelYear', 'ModelFamilydesc', 'EngineFuelType'], ['RepairType', 'ServiceType'] 
    
#     # Convert selected columns to string type
#     df[cat_cols1 + cat_cols2] = df[cat_cols1 + cat_cols2].astype(str)
    
#     if encoders is None:
#         encoder1, encoder2 = OneHotEncoder(sparse=False, sparse_output=False), OneHotEncoder(sparse=False, sparse_output=False)
    
#     else:
#         if isinstance(encoders, str):
#             # Load encoders from a file
#             encoders = load_encoders(encoders)
        
#         # Use the loaded encoders
#         encoder1, encoder2 = encoders['encoder1'], encoders['encoder2']

#     # Creating DataFrames from the encoded features with column names
#     encoded_df1 = pd.DataFrame(encoder1.fit_transform(df1[cat_cols1]), columns=encoder1.get_feature_names_out(cat_cols1))
#     encoded_df2 = pd.DataFrame(encoder2.fit_transform(df1[cat_cols2]), columns=encoder2.get_feature_names_out(cat_cols2))
    
#     # Concatenating the columns
#     df_new = pd.concat([df1.reset_index(drop=True), encoded_df1.reset_index(drop=True), encoded_df2.reset_index(drop=True)], axis=1)

#     # Categorical columns for aggregation
#     encoded_features1 = list(encoded_df1)
    
#     # Groupby columns
#     groupby_cols = ['Assetid', 'PONum'] + cat_cols1 + encoded_features1
    
#     # Convert the NumPy array to a list of column names or strings for encoded2
#     encoded2 = list(encoded_df2)
    
#     # Aggregation functions
#     agg_fn_encoded2 = {str(col): 'max' for col in encoded2}
#     agg_fn_others = {'BilledAmt': 'sum', 'MileageVal': 'mean'}
    
#     # Combining the dictionaries
#     agg_fn = {**agg_fn_encoded2, **agg_fn_others}
    
#     # Groupby to aggregate the columns 
#     df_new = df_new.groupby(groupby_cols).agg(agg_fn).reset_index()
    
#     return df_new

# Data Preparation
Clean up data convert the data into shape apt for Deep Learning model

In [204]:
def prepare_data(raw_data, sequence_length=3, for_prediction=False, fitted_scaler_mileage=None, fitted_scaler_amt=None, fitted_encoder=None):
    # Drop unnecessary columns
    cleaned_data = raw_data.drop(['MajorRepairInd', 'MinorRepairInd'], axis=1)
    # Define categorical columns
    cat_cols1, cat_cols2 = ['Manufacturername', 'ModelYear', 'ModelFamilydesc', 'EngineFuelType'], ['RepairType', 'ServiceType'] 

    # Convert selected columns to string type
    cleaned_data[cat_cols1 + cat_cols2] = cleaned_data[cat_cols1 + cat_cols2].astype(str)

    if fitted_encoder is None:
        encoder1 = OneHotEncoder(sparse=False, sparse_output=False)
        encoder2 = OneHotEncoder(sparse=False, sparse_output=False)
        encoder1.fit(cleaned_data[cat_cols1])
        encoder2.fit(cleaned_data[cat_cols2])

    else:
        encoders = fitted_encoder
        encoder1 = encoders['encoder1']
        encoder2 = encoders['encoder2']
        
    if fitted_scaler_mileage is None:
        scaler_mileage = MinMaxScaler()
        scaler_mileage.fit(cleaned_data[['MileageVal']])
        
    else:
        scaler_mileage = fitted_scaler_mileage

    if fitted_scaler_amt is None:
        scaler_amt = MinMaxScaler()
        scaler_amt.fit(cleaned_data[['BilledAmt']])
        
    else:
        scaler_amt = fitted_scaler_amt

    # Creating DataFrames from the encoded features with column names
    encoded_df1 = pd.DataFrame(encoder1.transform(cleaned_data[cat_cols1]), columns=encoder1.get_feature_names_out(cat_cols1))
    encoded_df2 = pd.DataFrame(encoder2.transform(cleaned_data[cat_cols2]), columns=encoder2.get_feature_names_out(cat_cols2))
    df_new = pd.concat([cleaned_data.reset_index(drop=True), encoded_df1.reset_index(drop=True), encoded_df2.reset_index(drop=True)], axis=1)

    # Categorical columns for aggregation
    encoded_features1 = list(encoded_df1)

    # Groupby columns
    groupby_cols = ['Assetid', 'PONum'] + cat_cols1 + encoded_features1
    
    # Convert the NumPy array to a list of column names or strings for encoded2
    encoded2 = list(encoded_df2)
    
    # Aggregation functions
    agg_fn_encoded2 = {str(col): 'max' for col in encoded2}
    agg_fn_others = {'BilledAmt': 'sum', 'MileageVal': 'mean'}
    
    # Combining the dictionaries
    agg_fn = {**agg_fn_encoded2, **agg_fn_others}
    
    # Groupby to aggregate the columns 
    df_new = df_new.groupby(groupby_cols).agg(agg_fn).reset_index()
    df_new = df_new.drop([ 'PONum', 'Manufacturername', 'ModelYear', 'ModelFamilydesc', 'EngineFuelType'],axis = 1)
    
    
    sequences = []
    labels = []

    for asset_id in df_new['Assetid'].unique():
        asset_data = df_new[df_new['Assetid'] == asset_id].sort_values(by='MileageVal')
        asset_data['MileageVal'] = scaler_mileage.transform(asset_data[['MileageVal']])
        asset_data['BilledAmt'] = scaler_amt.transform(asset_data[['BilledAmt']])
        asset_data_array = np.hstack([asset_data[['MileageVal', 'BilledAmt']].values, asset_data[df_new.columns[1:-2]].values])
        
        for i in range(len(asset_data_array) - sequence_length):
            sequence = asset_data_array[i:i + sequence_length]
            sequences.append(sequence)

            if not for_prediction:
                label = asset_data_array[i + sequence_length, 0]  # MileageVal is at index 0
                labels.append(label)

    sequences = np.array(sequences)

    if for_prediction:
        return sequences
    else:
        labels = np.array(labels)
        return sequences, labels, scaler_mileage, scaler_amt, encoder1, encoder2
  


In [205]:
raw_data = pd.read_excel(r'C:\Users\TLTUser\Desktop\AL\AL\Dataset\Dataset_LightTruck_15092023.xlsx', sheet_name='Sheet1')

In [206]:
fit_encoder(raw_data)
encoders = load_encoders('encoder.pkl')

sequences, labels, scaler_mileage, scaler_amt, encoder1, encoder2 = prepare_data(raw_data, fitted_encoder=encoders, sequence_length=3)
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

C:\Users\TLTUser\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\TLTUser\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [229]:
X_train_2d = X_train.reshape(-1, X_train.shape[-1])
X_test_2d = X_test.reshape(-1, X_test.shape[-1])

# Create dataframes for X_train and X_test with the same column names
column_names = [f'Column_{i+1}' for i in range(X_train_2d.shape[1])]

df_X_train = pd.DataFrame(X_train_2d, columns=column_names)
df_X_test = pd.DataFrame(X_test_2d, columns=column_names)

In [230]:
df_X_train.shape

(990, 64)

In [228]:
len(X_train)

330

In [209]:
# Convert the NumPy array to a Pandas DataFrame
df_test = pd.DataFrame(X_test.reshape(-1, X_test.shape[-1]), columns=[f'Column_{i+1}' for i in range(X_test.shape[-1])])

In [208]:
pd.set_option('display.max_column',None)
df_train.head()

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,Column_11,Column_12,Column_13,Column_14,Column_15,Column_16,Column_17,Column_18,Column_19,Column_20,Column_21,Column_22,Column_23,Column_24,Column_25,Column_26,Column_27,Column_28,Column_29,Column_30,Column_31,Column_32,Column_33,Column_34,Column_35,Column_36,Column_37,Column_38,Column_39,Column_40,Column_41,Column_42,Column_43,Column_44,Column_45,Column_46,Column_47,Column_48,Column_49,Column_50,Column_51,Column_52,Column_53,Column_54,Column_55,Column_56,Column_57,Column_58,Column_59,Column_60,Column_61,Column_62,Column_63,Column_64
0,0.442574,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.477734,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.498539,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.271985,0.144264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.272000,0.126598,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [184]:
original_column_2 = scaler_mileage.inverse_transform(df[['Column_2']])
df['original_billAmt'] = original_column_2

In [185]:
df.head()

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,Column_11,Column_12,Column_13,Column_14,Column_15,Column_16,Column_17,Column_18,Column_19,Column_20,Column_21,Column_22,Column_23,Column_24,Column_25,Column_26,Column_27,Column_28,Column_29,Column_30,Column_31,Column_32,Column_33,Column_34,Column_35,Column_36,Column_37,Column_38,Column_39,Column_40,Column_41,Column_42,Column_43,Column_44,Column_45,Column_46,Column_47,Column_48,Column_49,Column_50,Column_51,Column_52,Column_53,Column_54,Column_55,Column_56,Column_57,Column_58,Column_59,Column_60,Column_61,Column_62,Column_63,Column_64,original,original_miles,original_billAmt
0,0.119614,0.191368,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,47121.0,47121.0,75387.774491
1,0.121132,0.133206,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,47719.0,47719.0,52475.438375
2,0.121363,0.528645,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,47810.0,47810.0,208254.821426
3,0.121132,0.133206,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,47719.0,47719.0,52475.438375
4,0.121363,0.528645,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,47810.0,47810.0,208254.821426


# Build Model

In [187]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

In [188]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
9/9 [==============================] - 5s 109ms/step - loss: 0.0979 - val_loss: 0.0329
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0361 - val_loss: 0.0326
Epoch 3/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0293 - val_loss: 0.0198
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0239 - val_loss: 0.0160
Epoch 5/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0194 - val_loss: 0.0140
Epoch 6/50
9/9 [==============================] - 0s 10ms/step - loss: 0.0154 - val_loss: 0.0105
Epoch 7/50
9/9 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0092
Epoch 8/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0110 - val_loss: 0.0076
Epoch 9/50
9/9 [==============================] - 0s 13ms/step - loss: 0.0107 - val_loss: 0.0063
Epoch 10/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0096 - val_loss: 0.0054
Epoch 11/50
9/9 [===============

In [189]:
joblib.dump(filename ='model.pkl', value = model)

['model.pkl']

# Model Evaluation

In [190]:
from sklearn.metrics import r2_score

In [191]:
loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}')

3/3 [==============================] - 0s 4ms/step - loss: 0.0035
Test Loss: 0.0035074553452432156


In [192]:
y_pred_train = model.predict(X_train)

11/11 [==============================] - 1s 2ms/step


In [193]:
r2_train = r2_score(y_train,y_pred_train)
r2_train

0.9675788913993983

In [194]:
y_pred = model.predict(X_test)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)
r2

3/3 [==============================] - 0s 3ms/step


0.8705958699850562

# Sample Prediction

Replace new raw data for which predictions are to be made in place of sample_raw_data below

In [37]:
sample_sequences_for_prediction = prepare_data(raw_data[raw_data.Assetid==1888127],
                                               for_prediction=True,
                                               sequence_length=3,
                                               fitted_scaler_mileage=scaler_mileage,
                                               fitted_scaler_amt=scaler_amt,
                                               fitted_encoder=encoders)
print(sample_sequences_for_prediction.shape)
sample_prediction = model.predict(sample_sequences_for_prediction)


(3, 3, 64)
1/1 [==============================] - 1s 834ms/step


In [38]:
predictions_mileage = scaler_mileage.inverse_transform(sample_prediction)
predictions_mileage

array([[106556.695],
       [105425.77 ],
       [113331.086]], dtype=float32)

In [46]:
y_pred_all = scaler_mileage.inverse_transform(y_pred_all)
len(y_pred_all)               

413

In [195]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

In [196]:
# Train data prediction
y_pred_train = model.predict(X_train)
r2_score(y_train,y_pred_train)

11/11 [==============================] - 0s 3ms/step


0.9675788913993983

In [197]:
# Test data prediction
y_pred_test = model.predict(X_test)
r2_score(y_test,y_pred_test)

3/3 [==============================] - 0s 2ms/step


0.8705958699850562

In [218]:
X_train.shape

(330, 3, 64)

In [219]:
df_train.shape

(990, 65)

In [231]:
pred = model.predict(X_train)

11/11 [==============================] - 0s 3ms/step


In [233]:
len(pred)

330

In [279]:
flattened_array = X_train.reshape(X_train.shape[0], -1)
# Convert the flattened array to a DataFrame
df_train = pd.DataFrame(flattened_array)

In [280]:
flattened_array = X_test.reshape(X_test.shape[0], -1)
# Convert the flattened array to a DataFrame
df_test = pd.DataFrame(flattened_array)

In [281]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191
0,0.442574,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.477734,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.498539,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.271985,0.144264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.272000,0.126598,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.273127,0.137957,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.191800,0.143016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.194687,0.301771,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.197144,0.144749,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.344653,0.175402,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.357579,0.161187,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.364143,0.140781,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.472454,0.148132,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [282]:
y_pred_train = model.predict(X_train)

11/11 [==============================] - 0s 2ms/step


In [283]:
y_pred_train = scaler_mileage.inverse_transform(y_pred_train)

In [284]:
df_train['pred_miles'] = y_pred_train

In [285]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,pred_miles
0,0.442574,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.477734,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.498539,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,183795.375000
1,0.271985,0.144264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.272000,0.126598,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.273127,0.137957,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,108490.359375
2,0.191800,0.143016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.194687,0.301771,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.197144,0.144749,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,85417.960938
3,0.344653,0.175402,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.357579,0.161187,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.364143,0.140781,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,146629.640625
4,0.472454,0.148132,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [288]:
df_train['original_mailes'] = scaler_mileage.inverse_transform(df_train[[0]])

In [289]:
df_train .head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,pred_miles,original_mailes
0,0.442574,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.477734,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.498539,0.132163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,183795.375000,174348.0
1,0.271985,0.144264,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.272000,0.126598,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.273127,0.137957,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,108490.359375,107146.0
2,0.191800,0.143016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.194687,0.301771,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.197144,0.144749,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,85417.960938,75558.0
3,0.344653,0.175402,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.357579,0.161187,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.364143,0.140781,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,146629.640625,135773.0
4,0.472454,0.148132,0.0,

In [294]:
y_pred_test = model.predict(X_test)
y_pred_test = scaler_mileage.inverse_transform(y_pred_test)
df_test['pred_miles'] = y_pred_test

3/3 [==============================] - 0s 6ms/step


In [2]:
df_test['original_mailes'] = scaler_mileage.inverse_transform(df_test[[0]])
df_test.head(10)

NameError: name 'scaler_mileage' is not defined

## Service Type Prediction